In [186]:
import re
import os
import pandas as pd
from string import punctuation as punkt
from collections import defaultdict

import sqlite3
db = sqlite3.connect('OHG.db')
cursor = db.cursor()

In [187]:
cursor.execute('SELECT id FROM Tokens WHERE id_text = 13')
data = cursor.fetchall()
for i in range(0, len(data)):
    cursor.execute('UPDATE Tokens SET in_text = ? WHERE id = ?', (i, data[i-1][0]))
db.commit()

In [188]:
R_INDEX = defaultdict(lambda: defaultdict(set))
cursor.execute('SELECT * FROM Tokens')
T = cursor.fetchall()

In [190]:
T[10378]

(10379,
 10,
 630,
 6,
 'Hluduico',
 'hluduico',
 'hluduico',
 'luduiko',
 'luduiko',
 'ledic',
 'ledic',
 'L',
 None)

In [191]:
TtoT = {}
for i in T:
    try:
        TtoT[(i[1], i[3])] = (i[4], re.sub('<.*?>', '', i[12]))
    except:
        TtoT[(i[1], i[3])] = (i[4], i[12])

In [192]:
TtoT[(4, 4)]

('mit',
 'mit двн. свн. нн. (предл. с дат. [твор.] реже с вин.) с, по, одновременно с; дс. mið, med, да. mid, mið (в а. сохрани\xadлось только в midwife акушерка), ди. með, го. miþ; ср. дгр. metá.')

In [193]:
# "Tokens": 0 id_token, 1 id_text, 2 id_sent, 3 in_text, 
# 4 token, 5 lowered, 6 no_dia, 7 tran 8 no_d_tran 9 lemma 10 nod_lemma 11 lang 11 dict
for i in T:
    R_INDEX[i[5]][i[1]].add(i[3])
    R_INDEX[i[6]][i[1]].add(i[3])
    R_INDEX[i[7]][i[1]].add(i[3])
    R_INDEX[i[8]][i[1]].add(i[3])
    R_INDEX[i[9]][i[1]].add(i[3])
    R_INDEX[i[10]][i[1]].add(i[3])

In [194]:
R_INDEX['zungun']

defaultdict(set, {9: {258, 357, 967, 1032, 1058}})

In [195]:
def find_docs(words):
    all_indexes = [{} for _ in words]
    if len(words) > 0:
        result_t = set(R_INDEX[words[0]])
        all_indexes[0] = R_INDEX[words[0]]
        for key, i in enumerate(words[1:]):
            result_t = result_t & set(R_INDEX[i])
            all_indexes[key+1] = R_INDEX[i]
    else:
        result_t = set()
        
    #print (list(i.keys() for i in all_indexes))
    all_indexes = [{i:t[i] for i in t if i in result_t} for t in all_indexes]
    return result_t, all_indexes

In [196]:
R_INDEX['ioseph'], R_INDEX['iosepf']

(defaultdict(set, {7: {711, 742, 1092}}),
 defaultdict(set, {7: {711, 742, 1092}}))

In [197]:
R_INDEX = dict(R_INDEX)

In [198]:
for i in R_INDEX:
    R_INDEX[i] = dict(R_INDEX[i])

In [159]:
find_docs(['got','ist'])

({3, 5, 7, 9, 10},
 [{3: {315},
   5: {21, 191, 454, 721, 1537, 1674, 1712},
   7: {1038, 1254, 1581, 4068},
   9: {664, 1022, 1143, 1570, 1578},
   10: {152}},
  {3: {83, 282},
   5: {674,
    686,
    688,
    698,
    702,
    712,
    736,
    821,
    897,
    917,
    935,
    1087,
    1099,
    1308,
    1835,
    1906},
   7: {273,
    333,
    345,
    465,
    971,
    977,
    982,
    1125,
    1379,
    1419,
    1730,
    2023,
    2246,
    4085},
   9: {50,
    125,
    151,
    342,
    465,
    562,
    566,
    834,
    936,
    1000,
    1259,
    1268,
    1280,
    1287,
    1295,
    1302,
    1322,
    1332,
    1420,
    1464,
    1518,
    1571,
    1775,
    1944,
    1981,
    2036},
   10: {263}}])

In [160]:
def search(docs, indexes, begin=-10, end=10):
    total = defaultdict(list)
    for key, value in enumerate(docs):
        variants = indexes[0][value]
        for i in variants:
            area = set(range(max(0, begin+i), i+end+1))
            result = set([i])
            k = 1
            for i in indexes[1:]:
                overlap = area & i[value]
                if len(overlap) > 0:
                    result.update(overlap)
                    k += 1
                else:
                    break
            if k == len(indexes):
                total[value].append(result)
    return dict(total)

In [161]:
search(*find_docs(['got','ist']), begin=-10, end=10)

{5: [{712, 721}], 9: [{1570, 1571}, {1571, 1578}]}

In [162]:
search(*find_docs(['got','ist']), begin=-20, end=20)

{5: [{702, 712, 721, 736}], 7: [{4068, 4085}], 9: [{1570, 1571}, {1571, 1578}]}

In [199]:
TtoT[(4, 10)]

('ni',
 'ni, ne, свн. ne, en, in, част. отрицания не (у Нотк. и в более поздних памят\xadниках, а также в свн. пишется, как правило, вместе с глаголом); дс. ni, ne, да. ne, ди. né, рун. ni, го. ni.')

In [200]:
cursor.execute('SELECT id_text, title FROM Texts')
TEXTS = {i[0]:i[1] for i in cursor.fetchall()}
TEXTS

{1: 'Der Erste Merseburger Zauberspruch',
 2: 'Der Zweite Merseburger Zauberspruch',
 3: 'Das Hildebrandslied',
 4: 'Das Wessobrunner Gebet',
 5: 'Der althochdeutsche Isidor',
 6: 'Muspilli',
 7: 'Der althochdeutsche Tatian',
 8: 'Die Straßburger Eide',
 9: 'Otrfrids Evangelienbuch',
 10: 'Ludwigslied',
 11: 'Kasseler Glossen (Kasseler Gespräche)',
 12: 'Notker - Boethius Consolatio philosophiae',
 13: 'Der Ältere Physyologus'}

In [201]:
import json
import pickle as pkl

In [202]:
pkl.dump(TtoT, open('TtoT.pkl','wb'))
pkl.dump(T, open('T.pkl','wb'))
pkl.dump(R_INDEX, open('R_INDEX.pkl','wb'))
pkl.dump(TEXTS, open('TEXTS.pkl','wb'))

In [167]:
def interpreter(result):
    sentences = []
    for text in result:
        for item in result[text]:
            left = max(0, min(item) - 20)
            right = max(item) + 20
            sentence = ''
            for i in range(left, right+1):
                if (text, i,) in TtoT:
                    token = TtoT[(text, i,)]
                    if i in item:
                        tooltipped = '<a href="#" data-toggle="tooltip" title="{}"><b>{}</b> </a>'.format(token[1],token[0])
                        sentence = sentence + '<b>'+ tooltipped +'</b>' + ' '
                    else:
                        tooltipped = '<a href="#" data-toggle="tooltip" title="{}">{} </a>'.format(token[1],token[0])
                        sentence = sentence + tooltipped + ' '
            sentences.append((TEXTS[text], sentence,))
    return sentences

In [168]:
search(*find_docs(['got','ist']))

{5: [{712, 721}], 9: [{1570, 1571}, {1571, 1578}]}

In [169]:
def html_encoder(result):
    output = '<table border="2">'
    for i in result:
        output = output + '<tr><td>' + '</td><td>'.join(['<b>{}</b>'.format(i[0]), i[1]])+'</td></tr>'
    output += '</table>'
    return output

In [170]:
with open('doc.html','w') as f:
    R = interpreter(search(*find_docs(['ih'])))
    print (len(R))
    f.write(html_encoder(R))

60


In [171]:
interpreter(search(*find_docs(['ih'])))    

[('Das Hildebrandslied',
  '<a href="#" data-toggle="tooltip" title="ni, ne, свн. ne, en, in, част. отрицания не (у Нотк. и в более поздних памят\xadниках, а также в свн. пишется, как правило, вместе с глаголом); дс. ni, ne, да. ne, ди. né, рун. ni, го. ni.">ni </a> <a href="#" data-toggle="tooltip" title="gileiten (слгл. 1), см. leiten.\nleiten, gileiten (слгл. 1), свн. leiten на\xadправлять, вести, провожать; нн. lei\xadten, geleiten; дс. lêdian, да. lǣdan, ди. leiða; ср. двн. lîdan.">gileitos!» </a> <a href="#" data-toggle="tooltip" title="want срр одежда; материал (из которого сшита одежда); ср. нн. Gewand платье и Leinwand полотно.">want </a> <a href="#" data-toggle="tooltip" title="her, he (мест.), см. er.">her </a> <a href="#" data-toggle="tooltip" title="thoh, dhoh, doh, свн. doch (нар. сз.) однако, но, даже, также, хотя; нн. doch; дс. thoh, да. þēah, ди. þō, го. þauh.">do </a> <a href="#" data-toggle="tooltip" title="ar — Мусп.—(предл.), см. ur.\nur, ar, er, ir (предл.) от, из

In [46]:
find_docs(['got','ist'])

({3, 5, 7, 9, 10},
 [{3: {315},
   5: {21, 191, 454, 721, 1537, 1674, 1712},
   7: {1038, 1254, 1581, 4068},
   9: {664, 1022, 1143, 1570, 1578},
   10: {152}},
  {3: {83, 282},
   5: {674,
    686,
    688,
    698,
    702,
    712,
    736,
    821,
    897,
    917,
    935,
    1087,
    1099,
    1308,
    1835,
    1906},
   7: {273,
    333,
    345,
    465,
    971,
    977,
    982,
    1125,
    1379,
    1419,
    1730,
    2023,
    2246,
    4085},
   9: {50,
    125,
    151,
    342,
    465,
    562,
    566,
    834,
    936,
    1000,
    1259,
    1268,
    1280,
    1287,
    1295,
    1302,
    1322,
    1332,
    1420,
    1464,
    1518,
    1571,
    1775,
    1944,
    1981,
    2036},
   10: {263}}])

In [ ]:
def 

In [172]:
import json
json.dumps('aƕa',ensure_ascii=False)

'"aƕa"'

In [173]:
def find_docs(words):
    all_indexes = [{} for _ in words]
    if len(words) > 0:
        result_t = set(R_INDEX[words[0]])
        all_indexes[0] = R_INDEX[words[0]]
        for key, i in enumerate(words[1:]):
            result_t = result_t & set(R_INDEX[i])
            all_indexes[key+1] = R_INDEX[i]
    else:
        result_t = set()
        
    #print (list(i.keys() for i in all_indexes))
    all_indexes = [{i:t[i] for i in t if i in result_t} for t in all_indexes]
    return result_t, all_indexes

def search_doc(docs, indexes, begin=-10, end=10):
    total = defaultdict(list)
    for key, value in enumerate(docs):
        variants = indexes[0][value]
        for i in variants:
            area = set(range(max(0, begin+i), i+end+1))
            result = set([i])
            k = 1
            for i in indexes[1:]:
                overlap = area & i[value]
                if len(overlap) > 0:
                    result.update(overlap)
                    k += 1
                else:
                    break
            if k == len(indexes):
                total[value].append(result)
    return dict(total)

def interpreter(result):
    sentences = []
    for text in result:
        for item in result[text]:
            left = max(0, min(item) - 20)
            right = max(item) + 20
            sentence = ''
            for i in range(left, right+1):
                if (text, i,) in TtoT:
                    token = TtoT[(text, i,)]
                    if i in item:
                        if token[1] is not None:
                            tooltipped = '<a href="#" data-toggle="tooltip" title="{}"> <font size="5"> {}</font> </a>'.format(token[1],token[0])
                        else:
                            tooltipped = '<font size="5">{} </font>'.format(token[0])
                        sentence = sentence + '<b>'+ tooltipped.replace('\n','<br>') +'</b>' + ' '
                    else:
                        if token[1] is not None:
                            tooltipped = '<a href="#" data-toggle="tooltip" title="{}">{} </a>'.format(token[1],token[0])
                        else:
                            tooltipped = '{} '.format(token[0])
                        sentence = sentence + tooltipped.replace('\n','<br>') + ' '
            sentences.append((TEXTS[text], sentence,))
    return sentences

In [174]:
#"Tokens": 0 id_token, 1 id_text, 2 id_sent, 3 in_text, 4 token, 5 normalized, 6 lang 7 transitioned 8 dict
T[0]

(1,
 1,
 1,
 1,
 'Eiris',
 'eiris',
 'eiris',
 'eiris',
 'eiris',
 'eiris',
 'eiris',
 'D',
 'zierist, zeerest (нар.), свн. zerist, zerste (нар.) в первый раз, сначала; нн. zuerst.')

In [175]:
cursor.execute('SELECT * FROM Sentences')
S = cursor.fetchall()

In [176]:
S[0]

(1, ' ')

In [177]:
def norm(token, dic):
    if dic is not None:
        tooltipped = '<a href="#" data-toggle="tooltip" title="{}">{} </a>'.format((dic),token)
    else:
        tooltipped = '{} '.format(token)
    return tooltipped

In [178]:
t = 3
result = []
s = set()
for i in T:
    if i[1] == t:
        if i[2] not in s:
            par = ''
            for j in S:
                if j[0] == i[2]:
                    if j[1] == par or j[1] is None: 
                        par2 = ' '
                    else: 
                        par = j[1]
                        par2=par
            result.append([par2, norm(i[4], i[12])])
            s.add(i[2])
        else:
            result[-1][-1] = result[-1][-1] + norm(i[4], i[12])
            
html = '<table><tr><td width=10%>'
for i in result:
    html = html + i[0] + '</td><td>' + i[1]
    html += '<td></tr><tr><td>'
html += '</td></tr></table>'
html = html.replace('\n','<br>')

In [134]:
cursor.execute('SELECT id_sent, id FROM Tokens WHERE id_text = 3')
data = cursor.fetchall()

In [136]:
for i in data:
    cursor.execute('UPDATE Tokens SET id_sent = ? WHERE id = ?', (i[0]-72, i[1]))

In [137]:
db.commit()

In [179]:
with open ('./templates/Hildebrandslied.html'.lower(), 'w') as f:
    f.write('<style>\ntable {\nborder-style: no;\n}\n</style>{% extends "base.html" %}\n{% block content %}\n')
    f.write(html)
    f.write('\n{% endblock %}')

In [180]:
html

'<table><tr><td width=10%>1</td><td><a href="#" data-toggle="tooltip" title="ik дс. (мест.) я; ед. вин. mik, mt, mi, me, дат. mi, mi, me, род. mîn; двойств, им, wit, вин. дат. unk, род. unkero; мн. им, wî, wi, we, вин. дат, ûs, род. user; см. ih.">Ik </a><a href="#" data-toggle="tooltip" title="gihoran, gihören (слгл. 1), свн. gehoeren услышать, в свн. неперех. также при\xadнадлежать, подобать; нн. gehö\xadren.">gihorta </a><a href="#" data-toggle="tooltip" title="dat, that нжн., см. ther (daȥ).">dat </a><a href="#" data-toggle="tooltip" title="seggian дс. (слгл.): наст. изъяв. 1 л. ед. seggiu, мн. seggiad (по типу слгл. 1); см. sagên.">seggen, </a><td></tr><tr><td>2</td><td>ðat <a href="#" data-toggle="tooltip" title="sih, свн. нн. sich (возвр. мест.) вин. себя; в дс. да. отсутствует; ди. го. sik; ср. лат. se вин. абл. себя, собой, sibi дат. себе, русск. дат. «себе», вин. «себя».">sih </a><a href="#" data-toggle="tooltip" title="urhêtto — Хильт. 2— мр воин, витязь; да. ōretta.">urhett

In [62]:
TEXTS

{1: 'Der Erste Merseburger Zauberspruch',
 2: 'Der Zweite Merseburger Zauberspruch',
 3: 'Das Hildebrandslied',
 4: 'Das Wessobrunner Gebet',
 5: 'Der althochdeutsche Isidor',
 6: 'Muspilli',
 7: 'Der althochdeutsche Tatian',
 8: 'Die Straßburger Eide',
 9: 'Otrfrids Evangelienbuch',
 10: 'Ludwigslied',
 11: 'Kasseler Glossen (Kasseler Gespräche)',
 12: 'Notker - Boethius Consolatio philosophiae',
 13: 'Der Ältere Physyologus'}

##  Поиск по словарю

In [1]:
import json

In [2]:
with open('./ahd-dictionary/ohg_dict_final.html') as f:
    DICT = json.loads('['+','.join(f.readlines())+']')

In [3]:
def prettify(entry):
    entry = re.sub('<heading>', '<b>', entry)
    entry = re.sub('</heading>', '</b>', entry)
    entry = re.sub('<lang>', '<font color="red">', entry)
    entry = re.sub('</lang>', '</font>', entry)
    entry = re.sub('<lex>', '<font color="blue">', entry)
    entry = re.sub('</lex>', '</font>', entry)
    entry = re.sub('<.?(linked|gender|em)>', '', entry)
    entry = re.sub('<source>', '<font color="grey">', entry)
    entry = re.sub('</source>', '</font>', entry)
    return entry

In [4]:
dia = {'öøõôóōÖ':'o', 'íīî':'i', 'Äâáäā':'a', 'ēêḗёëéę':'e', 'úüūÜûù':'u', 'ýȳ':'y', 'ǽæǣ':'ae', 'œ':'oe','ȥ':'z'}

def no_dia(text):
    result = ''
    for i in text:
        if not re.match('[a-zA-Z]', i):
            found = False
            for j in dia:
                if i in j:
                    result += dia[j]
                    found = True
            if not found:
                result += i
        else:
            result += i
    return result

def lowered(text):
    return re.sub('[^\w]','',text.strip(punkt).lower())

In [5]:
cursor.execute('SELECT lemma, id FROM Tokens')
data = cursor.fetchall()

In [7]:
k = 0
m = 0
for i in data:
    if len(i[0].strip()) > 0:
        cursor.execute('UPDATE Tokens SET no_dia_lemma = ? WHERE id = ?', (no_dia(i[0]), i[1],))
        m += 1
    else:
        cursor.execute('UPDATE Tokens SET no_dia_lemma = ? WHERE id = ?', (i[0].strip(), i[1],))
        k += 1
k, m

(37, 12546)

In [8]:
db.commit()

In [5]:
def get_variance(entry):
    result = set()
    text = re.sub('<.*?>', '', entry.lower())
    text = re.sub('[^\w ]','',text)
    words = text.split()
    for word in words:
        result.add(no_dia(word))
        result.add(word)
    return result

In [8]:
prettify(DICT[0]['entry'])

'<b><strong>aba</strong></b>, <b><strong>abo</strong></b>, <font color="red">свн.</font> <font color="blue">abe</font> (нар.) вниз, прочь, далеко; <font color="blue">aba</font>, <font color="blue">abe</font>, <font color="blue">ab</font> (предл. с дат.) с, от, <font color="red">свн.</font> <font color="blue">abe</font>, <font color="blue">ab</font>; <font color="red">нн.</font> <font color="blue">ab</font>; <font color="red">дс.</font> <font color="red">ди.</font> <font color="red">го.</font> <font color="blue">af</font>, <font color="red">да.</font> <font color="red">а.</font> <font color="blue">of</font>.'

In [9]:
get_variance(DICT[1]['entry'])

{'2',
 'abahon',
 'abahôn',
 'воспринять',
 'ненавидеть',
 'неправильно',
 'отвращение',
 'слгл',
 'чувствовать'}

In [10]:
DICT_NODES = {}
DICT_INDEX = defaultdict(set)

for key, value in enumerate(DICT):
    DICT_NODES[key] = prettify(value['entry'])
    for i in get_variance(value['entry']):
        DICT_INDEX[i].add(key)

DICT_INDEX = dict(DICT_INDEX)

In [19]:
import pickle

In [21]:
pickle.dump(DICT_INDEX, open('DICT_INDEX.pkl','wb'))

In [22]:
pickle.dump(DICT_NODES, open('DICT_NODES.pkl','wb'))

In [13]:
def dict_search(words):
    words = re.sub('[^\w ]','',words).split()
    if len(words) > 0:
        result = DICT_INDEX[words[0]]
        for word in words:
            result = result & DICT_INDEX[word]
        result = [DICT_NODES[i] for i in result]
        return result
    else:
        return []

In [16]:
dict_search('рассматривать')

['<b><strong>brüeven</strong></b> <font color="red">свн.</font> (слгл.) взвешивать, рассмат\xadривать, испытывать; <font color="red">нн.</font> <font color="blue">prüfen</font>; (= <font color="red">стфр.</font> <font color="blue">prover</font>, <font color="red">лат.</font> <font color="blue">probare</font>).']

In [17]:
DICT[1]

{'headers': ['abahôn'],
 'entry': '<heading><strong>abahôn</strong></heading> (слгл. 2) <em>неправильно воспринять; ненавидеть, чувствовать отвращение</em>.',
 'sources': [],
 'other words': [],
 'linked words': [],
 'linked entries': []}